In [ ]:
import scvelo as scv
import numpy as np
import pandas as pd

import scanpy as sc
import matplotlib.pyplot as plt 

In [ ]:
adata = scv.datasets.pancreas()


In [ ]:
sc.pp.normalize_per_cell(adata)
sc.pp.log1p(adata)


# Foreign HVG

In [ ]:
hvg = np.loadtxt("./ZPSGenes.tsv",dtype=str)
hvg.shape

In [ ]:
filtered = adata[:,hvg]

In [ ]:
sc.pp.highly_variable_genes(adata,n_top_genes=4004)
adata.var['highly_variable']

In [ ]:
sc_calls = adata.var_names[adata.var['highly_variable']]
foreign_calls = adata.var_names[actual_mask]

In [ ]:
np.sum(actual_mask.astype(dtype=int))

In [ ]:
len(set(sc_calls).intersection(set(foreign_calls)))

# Vignette Analysis

In [ ]:
sc.pp.neighbors(filtered)
sc.tl.umap(filtered)
sc.pl.umap(filtered)

In [ ]:
scv.pp.moments(filtered,n_neighbors=None,n_pcs=None)

In [ ]:
scv.tl.velocity(filtered)
scv.tl.velocity_graph(filtered)


In [ ]:
scv.pl.velocity_embedding(filtered,figsize=(30,30))

# Velocity Graph 

In [ ]:
def delta_graph(origin,delta,highlight=[0,1],arrow_frequency=30,figsize=(10,8)):

    plt.figure(figsize=figsize)
    plt.scatter(*origin.T,color='red',s=1)

    for i,((x,y),(dx,dy)) in enumerate(zip(origin,delta)):
        if i%arrow_frequency == 0:
            plt.text(*origin[i],s=f"{i}")
            plt.arrow(x,y,dx,dy,color='green',head_width=.1,linewidth=.2)

    plt.show()

In [ ]:
pca_t0 = filtered.obsm['X_pca']

In [ ]:
# We want this rather than the velocity graph in itself
tg = np.array(scv.utils.get_transition_matrix(filtered).todense())

In [ ]:
def mean_velo(tg,data):
    wsum = np.matmul(tg,data) 
    norm_const = np.sum(tg,axis=1)
    wmean = (wsum.T / norm_const).T
    velo = wmean - data
    return velo
    # length = np.sqrt(np.sum(np.power(velo,2),axis=1))
    # unitized = (velo.T / length).T
    # return unitized

In [ ]:
pca_v = mean_velo(tg,pca_t0)

pca_t1 = pca_t0 + (pca_v / 5)

np.savetxt("sc_velo_pca_t0.tsv",pca_t0)
np.savetxt("sc_velo_pca_t1.tsv",pca_t1)

### Primitive UMAP velocity

In [ ]:
u_t0_primitive = filtered.obsm['X_umap']
u_t_v_primitive = mean_velo(tg,u_t0_primitive)

delta_graph(u_t0_primitive,u_t_v_primitive/5,arrow_frequency=5,figsize=(30,30))

# Foreign UMAP Train Transform 

In [ ]:
import matplotlib.pyplot as plt 

In [ ]:
from umap import UMAP
from sklearn.decomposition import PCA

In [ ]:
umap_model = UMAP(n_neighbors=15,min_dist=0.5, spread=1.0, n_components=2, negative_sample_rate=5, random_state=0,metric='cosine')
u_t0_transform = umap_model.fit_transform(pca_t0)

In [ ]:
u_t1_transform = umap_model.transform(pca_t1)

In [ ]:
u_t_v_transform = u_t1_transform - u_t0_transform

In [ ]:
plt.figure()
plt.scatter(*u_t0_transform.T,s=1)
plt.scatter(*u_t1_transform.T,s=1)
plt.show()

In [ ]:
delta_graph(u_t0_transform,u_t_v_transform,arrow_frequency=3,figsize=(30,30))

# Foreign UMAP Joint

In [ ]:
def umap_velocity_via_joint(t0,t1):
    stacked = np.vstack([t0,t1])
    
    umap_model = UMAP(n_neighbors=15,min_dist=0.5, spread=1.0, n_components=2, negative_sample_rate=5, random_state=0,metric='cosine')
    u_t_joint = umap_model.fit_transform(stacked)
    
    u_t0 = u_t_joint[:t0.shape[0]]
    u_t1 = u_t_joint[t0.shape[0]:]
    
    u_t_v = u_t1 - u_t0
    return u_t0,u_t1,u_t_v

u_t0_joint,u_t1_joint,u_t_v_joint = umap_velocity_via_joint(pca_t0,pca_t1)

# stacked = np.vstack([pca_t0,pca_t1])

# umap_model = UMAP(n_neighbors=15,min_dist=0.5, spread=1.0, n_components=2, negative_sample_rate=5, random_state=0,metric='cosine')
# u_t_joint = umap_model.fit_transform(stacked)

# u_t0_joint = u_t_joint[:pca_t0.shape[0]]
# u_t1_joint = u_t_joint[pca_t0.shape[0]:]

# u_t_v_joint = u_t1_joint - u_t0_joint

In [ ]:
plt.figure()
plt.scatter(*u_t0_joint.T,s=1)
plt.scatter(*u_t1_joint.T,s=1)
plt.show()

In [ ]:
delta_graph(u_t0_joint,u_t_v_joint,arrow_frequency=3,figsize=(30,30))

# Let's compare smoothness in several situations 

In [ ]:
from scipy.spatial.distance import pdist,squareform

def local_velocity_smoothness(velocities,knn):
    smoothness = []
    for neighborhood in knn:
        local_velocities = velocities[neighborhood]
        local_smoothness = np.mean(pdist(local_velocities,metric='cosine'))
        smoothness.append(local_smoothness)
    return smoothness

def extract_knn_from_adata(adata,k=None):

    samples = adata.shape[0]
    knn = []
    
    for i in range(samples):
        conn = np.array(adata.obsp['connectivities'][i].todense()).flatten()
        mask = conn > 0
        sort = np.argsort(conn[mask])
        indices = list(np.arange(samples)[mask][sort])
        knn.append(indices)
    
    clean = np.min([len(ragged) for ragged in knn])
    if k is None:
        k = clean
    else:
        if k > clean:
            print(f"WARNING: k connectivities aren't available everywhere. Setting to minimum k={clean}")
            k = clean
            
    knn = [ragged[-k:] for ragged in knn]
    return knn

knn = extract_knn_from_adata(filtered,k=5)

In [ ]:
# velocity according to scvelo
sc_velo_smoothness = local_velocity_smoothness(filtered.layers['velocity'],knn)
# plt.figure()
# plt.hist(sc_velo_smoothness,bins=30)
# plt.show()

plt.figure(figsize=(12,10))
plt.scatter(*u_t0_primitive.T,c=sc_velo_smoothness)
plt.colorbar()
plt.show()

In [ ]:
# sc velo l2 velocity
sc_velo_velo = np.linalg.norm(np.array(filtered.layers['velocity']),axis=1)

plt.figure(figsize=(12,10))
plt.scatter(*u_t0_primitive.T,c=sc_velo_velo)
plt.colorbar()
plt.show()


In [ ]:
# PCA velocity 
pca_velo_smoothness = local_velocity_smoothness(pca_v,knn)
# plt.figure()
# plt.hist(pca_velo_smoothness,bins=30)
# plt.show()


plt.figure(figsize=(12,10))
plt.scatter(*u_t0_primitive.T,c=pca_velo_smoothness)
plt.colorbar()
plt.show()

In [ ]:
# Umap velocity using umap directly with a graph 

In [ ]:
u_t_primitive_smoothness = local_velocity_smoothness(u_t_v_primitive,knn)

plt.figure(figsize=(12,10))
plt.scatter(*u_t0_primitive.T,c=u_t_primitive_smoothness)
plt.colorbar()
plt.show()

In [ ]:
u_t_joint_smoothness = local_velocity_smoothness(u_t_v_joint,knn)

plt.figure(figsize=(12,10))
plt.scatter(*u_t0_primitive.T,c=u_t_joint_smoothness)
plt.colorbar()
plt.show()

In [ ]:
u_t_v_joint.shape

# Predicted Foreign Umap

In [ ]:
t_1 = np.loadtxt("./trajectories_t1.tsv",dtype=float).T
t_2 = np.loadtxt("./trajectories_t2.tsv",dtype=float).T
t_3 = np.loadtxt("./trajectories_t3.tsv",dtype=float).T
t_4 = np.loadtxt("./trajectories_t4.tsv",dtype=float).T
t_5 = np.loadtxt("./trajectories_t5.tsv",dtype=float).T
t_6 = np.loadtxt("./trajectories_t6.tsv",dtype=float).T


In [ ]:
def arrow_deltas(t1,t2):
    x,y = t1
    dx,dy = t2 - t1
    return (x,y,dx,dy)

def trajectory_series(trajectories,frequency=5):
    plt.figure(figsize=(30,30))
    plt.scatter(*trajectories[0].T,s=3)
    for t1,t2 in zip(trajectories[:-1],trajectories[1:]):
        for i in range(0,t1.shape[0],frequency): 
            plt.arrow(*arrow_deltas(t1[i],t2[i]),color='green',head_width=.05,linewidth=.2)
    plt.show()    

trajectory_series([u_t + 5,t_1,t_2,t_3,t_4,t_5,t_6])
    
# plt.figure(figsize=(30,30))
# plt.scatter(*u_t.T + 5,s=1)
# for i in range(0,trajectories_1.shape[0],30): 
#     plt.arrow(*arrow_deltas(u_t[i] + 5,trajectories_1[i]),color='red',head_width=.1,linewidth=.2)
#     plt.arrow(*arrow_deltas(trajectories_1[i],trajectories_2[i]),color='green',head_width=.1,linewidth=.2)
#     plt.arrow(*arrow_deltas(trajectories_2[i],trajectories_3[i]),color='blue',head_width=.1,linewidth=.2)
# plt.show()

In [ ]:
np.max(t_5,axis=0)

In [ ]:
# torubleshooting offsets

# some trajectory starts at ~ 4,10.5
# First let's index it, it should be easy to find 

t1


# Joint Embedding?

In [ ]:
t1 = np.array(filtered.X.copy().todense())

# # Is this in log space?
v = np.array(filtered.layers['velocity'])
v = np.log1p(np.abs(v)) * np.sign(v)

t2 = t1+(v/5)

# clip negatives

t2[t2 < 0] = 0

In [ ]:
from umap import UMAP
from sklearn.decomposition import PCA

def arrow_deltas(time1,time2,damping=1):
    x,y = time1
    dx,dy = time2 - time1
    return (x,y,dx*damping,dy*damping)
    
def simple_sequence(time1,time2,highlight=[0,1],arrow_frequency=30):
    pca_model = PCA(n_components=50)
    pca_t1 = pca_model.fit_transform(time1)
    pca_t2 = pca_model.transform(time2)
    
    umap_model = UMAP(n_neighbors=15,min_dist=0.5, spread=1.0, n_components=2, negative_sample_rate=5, random_state=0,metric='cosine')
    u_t1 = umap_model.fit_transform(pca_t1)
    u_t2 = umap_model.transform(pca_t2)

    plt.figure(figsize=(10,8))
    plt.scatter(*u_t1.T,color='red',s=1)
    # plt.show()
    # plt.figure(figsize=(10,8))
    plt.scatter(*u_t2.T,color='blue',s=1)
    plt.show()

    # plt.figure(figsize=(10,8))
    # plt.scatter(*u_t1.T,s=1)
    # for i in highlight:
    #     plt.text(*u_t1[i],s=f"{i}")
    #     plt.text(*u_t2[i],s=f"{i}'")
    #     plt.arrow(*arrow_deltas(u_t1[i],u_t2[i]),color='red',head_width=.1,linewidth=.2)
    # for i in range(0,u_t1.shape[0],arrow_frequency): 
    #     plt.arrow(*arrow_deltas(u_t1[i],u_t2[i]),color='green',head_width=.1,linewidth=.2)
    # plt.show()

    return u_t1
    
    # plt.figure(figsize=(10,8))
    # plt.scatter(*pca_t1[:,:2].T,s=1)
    # for i in highlight:
    #     plt.text(*pca_t1[i,:2],s=f"{i}")
    # for i in range(0,pca_t1[:,:2].shape[0],arrow_frequency): 
    #     plt.arrow(*arrow_deltas(pca_t1[i,:2],pca_t2[i,:2],damping=1),color='green',head_width=.1,linewidth=.2)
    # plt.show()